In [1]:
import pandas as pd
import numpy as np

# Load source data
file_path = r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Overhead Report\Dump Data Automation\Overhead Report-SAP Download.xlsx"
df = pd.read_excel(file_path)

mapping_path = r"Y:\Admin - General\Finance\Monthly Accounts\2025\Dump Data Automation\COAS.xlsx"

df_mapping = pd.read_excel(mapping_path)
df_mapping=df_mapping[['Order', 'Description', 'Responsible CCtr' ]]

#Read IPT to replace cost center that contains 'IPT' with Fright Brand
temp_path=r"Y:\Admin - General\Finance\Monthly Accounts\2025\Dump Data Automation\Overhead Report-Dump Data Template.xlsx"
df_IPT1=pd.read_excel(temp_path, sheet_name='IPT Mapping1')
df_IPT2=pd.read_excel(temp_path, sheet_name='IPT Mapping2')

# Keep only unique Order mappings
df_mapping = df_mapping.drop_duplicates(subset='Order')

#create 'Ori Cost Center' to keep orginial cost center info for reference
df['Ori Cost Center']=df['Cost Center']

# Merge on Order
df_final = df.merge(df_mapping, on='Order', how='left')

# Fill missing Cost Center from Responsible CCtr, Fill Blank Text column with Description
if 'Responsible CCtr' in df_final.columns:
    df_final['Cost Center'] = df_final['Cost Center'].fillna(df_final['Responsible CCtr'])
    df_final['Text']=df_final['Text'].fillna(df_final['Description'])
    df_final.drop(columns='Responsible CCtr', inplace=True, errors='ignore')



In [2]:
#---------------------------------------------------Replace IPT-----------------------------------------
# Merge with df_IPT1
df_final = df_final.merge(df_IPT1, on=['G/L Account','Cost Center'], how='left')

# Step 1: Define the 2 G/L Accounts of interest
gl_accounts = ['6142000010', '6142000060']

# Step 2: Convert 'G/L Account' to string (just in case it's numeric)
df_final['G/L Account'] = df_final['G/L Account'].astype(str)
df_final['Cost Center'] = df_final['Cost Center'].astype(str)
# Ensure consistent string types across all dataframes
df_IPT1['G/L Account'] = df_IPT1['G/L Account'].astype(str)
df_IPT1['Cost Center'] = df_IPT1['Cost Center'].astype(str)
df_IPT2['G/L Account'] = df_IPT2['G/L Account'].astype(str)
df_final['Plant'] = pd.to_numeric(df_final['Plant'], errors='coerce').astype('Int64')
df_IPT2['Plant'] = pd.to_numeric(df_IPT2['Plant'], errors='coerce').astype('Int64')

# Step 3: Apply your rules
# Scenario 1 & 2: If G/L Account is in target and Cost Center contains 'IPT1' → replace with IPT column, based on Cost Center
df_final.loc[
    (df_final['G/L Account'].isin(gl_accounts)) &
    (df_final['Cost Center'].astype(str).str.contains('IPT', na=False)),
    'Cost Center'
] = df_final['IPT1']

# Convert string 'nan' to real NaN
df_final['Cost Center'] = df_final['Cost Center'].replace('nan', pd.NA)
df_final['Cost Center'] = df_final['Cost Center'].replace(['', 'nan'], pd.NA)

# Scenario 3: If Cost Center is blank or nan and G/L Account is in target → fill with IPT2 column, based on Plant
#First merge with IPT2
# Merge on 'G/L Account' and 'Plant'
df_final = df_final.merge(df_IPT2, on=['G/L Account', 'Plant'], how='left')

# Replace only rows where Cost Center is blank AND G/L Account is in target list
mask = (
    df_final['G/L Account'].isin(gl_accounts) &
    df_final['Cost Center'].isna()
)

df_final.loc[mask, 'Cost Center'] = df_final.loc[mask, 'IPT2']

# Scenario 4: If Cost Center and Plant are both blank → fill with 'Freight Siniat'
df_final.loc[
    (df_final['G/L Account'].isin(gl_accounts)) & df_final['Cost Center'].isna() & df_final['Plant'].isna(),
    'Cost Center'
] = 'Freight Siniat'


In [3]:
#-------------------------Remove error document rows and pairing rows---------------------
# Step 1: Read the list of document numbers to check
df_doc2remove = pd.read_excel(temp_path, sheet_name='Remove Doc')
doc_nums = df_doc2remove['Document Number'].dropna().unique()

# Step 2: Filter df_final to get only the relevant rows
rows_to_check = df_final[df_final['Document Number'].isin(doc_nums)]

# Step 3: Calculate total amount for those document numbers
total_amount = rows_to_check['Amount in Local Currency'].sum()

# Step 4: Remove the rows only if the total adds up to zero
if abs(total_amount) < 1e-6:  # small threshold to account for rounding
    df_final = df_final[~df_final['Document Number'].isin(doc_nums)]
    print("Rows removed successfully.")
else:
    print("Total documents do not sum to zero — no rows removed.")

# create 'Change' column
df_final['Change'] = np.where(
    df_final['Ori Cost Center']!= df_final['Cost Center'], 
    df_final['Cost Center'], 
    ''
)

# Select columns for export
df_final = df_final[[ 
    'Document Number', 'Document Date', 'Posting Date', 'Entry Date', 'User Name',
    'Cost Center', 'G/L Account', 'Tax Code', 'Amount in Local Currency', 'Text',
    'Order', 'Assignment', 'Trading Partner No.', 'WBS Element', 'Document Currency', 
    'Amount in Doc. Curr.','Plant','Purchasing Document','Supplier','Ori Cost Center','Change'
]]

# Format dates to dd/mm/yyyy
for date_col in ['Document Date', 'Posting Date', 'Entry Date']:
    df_final[date_col] = pd.to_datetime(df_final[date_col], errors='coerce').dt.strftime('%d/%m/%Y')



Rows removed successfully.


In [4]:
# Load the workbook
from openpyxl import load_workbook
wb = load_workbook(temp_path)
ws = wb["SAP Data Final"]

# Clear old data but keep headers
ws.delete_rows(2, ws.max_row)  # Deletes from row 2 downwards (keeps headers)

# Replace <NA> with None for Excel compatibility
df_final = df_final.applymap(lambda x: None if pd.isna(x) else x)

# Write updated DataFrame to "SAP Data Final" (including headers)
for c_idx, column in enumerate(df_final.columns, start=1):
    ws.cell(row=1, column=c_idx, value=column)  # Write column headers

for r_idx, row in enumerate(df_final.itertuples(index=False), start=2):  # Start from row 2 (below headers)
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save the workbook
wb.save(temp_path)

C:\Users\141823\AppData\Local\Temp\ipykernel_27236\1863714481.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_final = df_final.applymap(lambda x: None if pd.isna(x) else x)
